In [1]:
import pandas as pd
from pathlib import Path
data_root = Path('../kkdata3/')
train_source = pd.read_parquet(data_root / 'label_train_source.parquet')
train_target = pd.read_parquet(data_root / 'label_train_target.parquet')
test_source = pd.read_parquet(data_root / 'label_test_source.parquet')
meta_song = pd.read_parquet(data_root / 'meta_song.parquet')
#meta_song_composer = pd.read_parquet(data_root / 'meta_song_composer.parquet')
#meta_song_genre = pd.read_parquet(data_root / 'meta_song_genre.parquet')
#meta_song_lyricist = pd.read_parquet(data_root / 'meta_song_lyricist.parquet')
#meta_song_producer = pd.read_parquet(data_root / 'meta_song_producer.parquet')
#meta_song_titletext = pd.read_parquet(data_root / 'meta_song_titletext.parquet')


train_source.dtypes

session_id          int64
song_id            object
unix_played_at      int64
play_status         int64
login_type          int64
listening_order     int64
dtype: object

In [5]:
train_all = pd.concat([train_source, train_target], axis=0, ignore_index=True)
train_all

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,751,6027767fad949f3ca5e772df04924949,1659598909,1,7,1
1,751,041547bddb0a3e730f32db84c65868ca,1659599182,1,7,2
2,751,041547bddb0a3e730f32db84c65868ca,1659599184,0,7,3
3,751,041547bddb0a3e730f32db84c65868ca,1659599185,0,7,4
4,751,8b32f88104ecf859be934d9b45f30cd1,1659599218,1,7,5
...,...,...,...,...,...,...
14306470,714580,55728030c86e1afb7aa4a31fee56d3b8,1666643323,0,7,21
14306471,714580,741590eee67a948a1b8b3a9ff60682bf,1666643442,0,7,22
14306472,714580,3c90623f4a5995b59a4f4d4eafcde4cc,1666643515,0,7,23
14306473,714580,4a3c3bf3793672422bc87ee280087315,1666643581,0,7,24


In [6]:
del train_source, train_target

In [7]:
# map song_id to song_index to save memory and speed up 
meta_song['song_index'] = meta_song.index
train_all = train_all.merge(meta_song[['song_id', 'song_index']], on='song_id', how='left')
test_source = test_source.merge(meta_song[['song_id', 'song_index']], on='song_id', how='left')
del train_all['song_id']
del test_source['song_id']

In [8]:
train_all['previous1_song_index'] = train_all.groupby('session_id')['song_index'].shift(1)
train_all['previous2_song_index'] = train_all.groupby('session_id')['song_index'].shift(2)
test_source['previous1_song_index'] = test_source.groupby('session_id')['song_index'].shift(1)
test_source['previous2_song_index'] = test_source.groupby('session_id')['song_index'].shift(2)
history_fields = ['previous1_song_index', 'previous2_song_index', 'song_index']
history=pd.concat([train_all[history_fields], test_source[history_fields]], ignore_index=True)
history.dropna(inplace=True)
history

,previous1_song_index,previous2_song_index,song_index
2,753834.0,283767.0,753834
3,753834.0,753834.0,753834
4,753834.0,753834.0,955400
5,955400.0,753834.0,13609
6,13609.0,955400.0,955400
...,...,...,...
17167750,154541.0,125431.0,408927
17167751,408927.0,154541.0,125430
17167752,125430.0,408927.0,490246
17167753,490246.0,125430.0,482103


In [9]:
gram1 = history[['previous1_song_index', 'song_index']].groupby(['previous1_song_index']).value_counts(normalize=True, sort=True)
gram1 = gram1.reset_index().groupby('previous1_song_index').first().reset_index()
gram1

,previous1_song_index,song_index,proportion
0,1.0,1014814,1.000000
1,3.0,87846,0.333333
2,4.0,136296,0.111111
3,5.0,5,0.131579
4,6.0,519864,1.000000
...,...,...,...
748407,1030707.0,457109,0.083333
748408,1030708.0,736280,0.212291
748409,1030709.0,915034,0.046083
748410,1030710.0,1030649,0.955556


In [10]:
gram2 = history[['previous2_song_index', 'previous1_song_index', 'song_index']].groupby(['previous2_song_index', 'previous1_song_index']).value_counts(normalize=True, sort=True)
gram2 = gram2.reset_index().groupby(['previous2_song_index','previous1_song_index']).first().reset_index()
gram2

,previous2_song_index,previous1_song_index,song_index,proportion
0,1.0,488417.0,697989,1.0
1,1.0,1014814.0,799780,1.0
2,3.0,87846.0,897082,1.0
3,3.0,200709.0,1021838,1.0
4,3.0,401307.0,318166,1.0
...,...,...,...,...
9870459,1030711.0,936567.0,1023129,1.0
9870460,1030711.0,946599.0,125355,1.0
9870461,1030711.0,967484.0,609494,1.0
9870462,1030711.0,967508.0,561432,1.0


In [11]:
gram1.to_parquet('gram1.parquet')
gram2.to_parquet('gram2.parquet')

In [27]:
df = test_source.pivot_table(index='session_id', columns='listening_order', values='song_index', aggfunc='first', fill_value=0)

In [30]:
df.reset_index(inplace=True)
df

listening_order,session_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,8,409114,14164,583221,692846,825145,954663,997977,801730,907091,...,898874,714971,761849,743151,722938,893887,935947,991557,787899,966227
1,9,994817,476351,740369,53552,393840,769569,551759,606304,845032,...,592133,959109,946504,896683,413950,1025451,319983,607582,80538,950108
2,18,466606,466606,926225,165647,892006,369362,17868,109937,190807,...,268408,268408,554079,211614,551759,268408,190142,305233,653191,345306
3,19,369362,226980,62714,173342,643147,50080,641579,416570,173340,...,128037,413849,416570,329393,151316,173340,558191,162042,173340,233417
4,28,873514,754277,873514,933943,873514,172661,172661,307978,284339,...,917177,972089,714517,1003654,713500,844786,191916,780583,714127,107386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143059,715299,206568,112601,625135,526899,625135,206568,112601,491258,491258,...,318406,762442,958625,854691,566286,967761,287336,922316,718430,111963
143060,715308,467938,663060,17868,17868,98610,328687,213625,137670,443142,...,700323,75262,454993,154012,521272,551592,310226,87473,442960,14658
143061,715309,443142,24796,425434,439189,599361,223714,309404,553465,408937,...,48134,198516,893649,89141,905307,353728,688233,865220,411543,87826
143062,715318,872433,873192,895819,999507,892947,863050,755735,898646,938634,...,778748,1000587,935084,792764,717112,981883,201902,859635,336383,395865


In [31]:
gram1_predict = pd.merge(df, gram1, left_on=[20], right_on=['previous1_song_index'], how='left')

In [32]:
gram1_predict['p'] = gram1_predict['song_index'].fillna(gram1_predict[20]).astype(int).values

In [33]:
del df

In [34]:
for k in ['song_index', 'previous1_song_index', 'proportion']:
    del gram1_predict[k]
gram1_predict

,session_id,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,p
0,8,409114,14164,583221,692846,825145,954663,997977,801730,907091,...,714971,761849,743151,722938,893887,935947,991557,787899,966227,1015754
1,9,994817,476351,740369,53552,393840,769569,551759,606304,845032,...,959109,946504,896683,413950,1025451,319983,607582,80538,950108,926212
2,18,466606,466606,926225,165647,892006,369362,17868,109937,190807,...,268408,554079,211614,551759,268408,190142,305233,653191,345306,345306
3,19,369362,226980,62714,173342,643147,50080,641579,416570,173340,...,413849,416570,329393,151316,173340,558191,162042,173340,233417,177048
4,28,873514,754277,873514,933943,873514,172661,172661,307978,284339,...,972089,714517,1003654,713500,844786,191916,780583,714127,107386,107386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143059,715299,206568,112601,625135,526899,625135,206568,112601,491258,491258,...,762442,958625,854691,566286,967761,287336,922316,718430,111963,111963
143060,715308,467938,663060,17868,17868,98610,328687,213625,137670,443142,...,75262,454993,154012,521272,551592,310226,87473,442960,14658,169541
143061,715309,443142,24796,425434,439189,599361,223714,309404,553465,408937,...,198516,893649,89141,905307,353728,688233,865220,411543,87826,619983
143062,715318,872433,873192,895819,999507,892947,863050,755735,898646,938634,...,1000587,935084,792764,717112,981883,201902,859635,336383,395865,336383


In [38]:
gram2_predict = pd.merge(gram1_predict, gram2, left_on=[19,20], right_on=['previous2_song_index', 'previous1_song_index'], how='left')
for k in ['previous1_song_index', 'previous2_song_index', 'proportion']:
    del gram2_predict[k]
gram2_predict

,session_id,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,p,song_index
0,8,409114,14164,583221,692846,825145,954663,997977,801730,907091,...,761849,743151,722938,893887,935947,991557,787899,966227,1015754,715921.0
1,9,994817,476351,740369,53552,393840,769569,551759,606304,845032,...,946504,896683,413950,1025451,319983,607582,80538,950108,926212,NaN
2,18,466606,466606,926225,165647,892006,369362,17868,109937,190807,...,554079,211614,551759,268408,190142,305233,653191,345306,345306,571377.0
3,19,369362,226980,62714,173342,643147,50080,641579,416570,173340,...,416570,329393,151316,173340,558191,162042,173340,233417,177048,978997.0
4,28,873514,754277,873514,933943,873514,172661,172661,307978,284339,...,714517,1003654,713500,844786,191916,780583,714127,107386,107386,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143059,715299,206568,112601,625135,526899,625135,206568,112601,491258,491258,...,958625,854691,566286,967761,287336,922316,718430,111963,111963,NaN
143060,715308,467938,663060,17868,17868,98610,328687,213625,137670,443142,...,454993,154012,521272,551592,310226,87473,442960,14658,169541,NaN
143061,715309,443142,24796,425434,439189,599361,223714,309404,553465,408937,...,893649,89141,905307,353728,688233,865220,411543,87826,619983,619983.0
143062,715318,872433,873192,895819,999507,892947,863050,755735,898646,938634,...,935084,792764,717112,981883,201902,859635,336383,395865,336383,7803.0


In [39]:
del gram1_predict

In [40]:
gram2_predict['p2'] = gram2_predict['song_index'].fillna(gram2_predict['p']).astype(int)

In [41]:
cond = (gram2_predict[19] == gram2_predict[20])
gram2_predict['p3'] = gram2_predict['p2'].where(~cond, gram2_predict[20])

In [43]:
gram2_predict

,session_id,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,p,song_index,p2,p3
0,8,409114,14164,583221,692846,825145,954663,997977,801730,907091,...,722938,893887,935947,991557,787899,966227,1015754,715921.0,715921,715921
1,9,994817,476351,740369,53552,393840,769569,551759,606304,845032,...,413950,1025451,319983,607582,80538,950108,926212,NaN,926212,926212
2,18,466606,466606,926225,165647,892006,369362,17868,109937,190807,...,551759,268408,190142,305233,653191,345306,345306,571377.0,571377,571377
3,19,369362,226980,62714,173342,643147,50080,641579,416570,173340,...,151316,173340,558191,162042,173340,233417,177048,978997.0,978997,978997
4,28,873514,754277,873514,933943,873514,172661,172661,307978,284339,...,713500,844786,191916,780583,714127,107386,107386,NaN,107386,107386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143059,715299,206568,112601,625135,526899,625135,206568,112601,491258,491258,...,566286,967761,287336,922316,718430,111963,111963,NaN,111963,111963
143060,715308,467938,663060,17868,17868,98610,328687,213625,137670,443142,...,521272,551592,310226,87473,442960,14658,169541,NaN,169541,169541
143061,715309,443142,24796,425434,439189,599361,223714,309404,553465,408937,...,905307,353728,688233,865220,411543,87826,619983,619983.0,619983,619983
143062,715318,872433,873192,895819,999507,892947,863050,755735,898646,938634,...,717112,981883,201902,859635,336383,395865,336383,7803.0,7803,7803


In [59]:
predictions = []
available_songs = set(meta_song['song_index'])
available_songs = available_songs - set(gram2_predict['p3'])
src_cols = list(range(20,0,-1))
for i, l in gram2_predict.iterrows():
    predict = []
    record = {'session_id': l['session_id'],}
    predict.append(l['p3'])
    src_songs = set(l[src_cols])
    if src_songs & available_songs:
        for i in l[src_cols]:
            if i in available_songs:
                predict.append(i)
                available_songs.remove(i)
                if len(predict) == 5:
                    break
    while len(predict) < 5:
        predict.append(available_songs.pop())
    break